In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACE_API_TOKEN")


In [2]:
# Read the pdfs 
loader = PyPDFDirectoryLoader("./us_census")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(documents)
documents[0]

Document(page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015Issued September 2023Douglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to health coverage. For example, between 2021 and 2022, the labor market continued to improve, which may have affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under the Continuous Enrollment Provision.\n2 The American \nRescue Plan (ARP) enhanced Marketplace premium subsidies for those with incomes above 400 percent of the poverty level as well as for unemployed people.\n3', metadata={'source': 'us_census\\acsbr-015.pdf', 'page': 0})

In [3]:
len(documents)

316

In [4]:
# Embeddings using Huggingface
hugggingface_embeddings = HuggingFaceBgeEmbeddings(
    # model="BAAI/bge-small-en-v1.5",
    # model_kwargs={"device":"cpu"},
    # encode_kwargs={"normalize_embeddings":True}
)

e:\projects\langchain_projects\base\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
e:\projects\langchain_projects\base\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
e:\projects\langchain_projects\base\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
# hugggingface_embeddings.embed_query(documents[0].page_content)

In [8]:
# Vector store creation
vector_db = FAISS.from_documents(documents[:120], hugggingface_embeddings)

In [9]:
# Query using similarity search
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vector_db.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [10]:
retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [14]:
from langchain_community.llms import HuggingFaceHub

hf = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.1,"max_length":500}   
)
query = "What is the health insurance coverage?"
hf.invoke(query)

HfHubHTTPError: 429 Client Error: Too Many Requests for url: https://api-inference.huggingface.co/models/mistralai/Mistral-7B-v0.1 (Request ID: aIrifShpfM8kJfhwXV5o4)

Rate limit reached. You reached free usage limit (reset hourly). Please subscribe to a plan at https://huggingface.co/pricing to use the API at this rate

In [17]:
#Hugging Face models can be run locally through the HuggingFacePipeline class.
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(
    # model_id="mistralai/Mistral-7B-v0.1",
    model_id="gpt2",
    task="text-generation",
    pipeline_kwargs={"temperature": 0.1, "max_new_tokens": 300}
)

llm.invoke(query)

e:\projects\langchain_projects\base\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


'What is the health insurance coverage?\n\nThe health insurance coverage is a form of insurance that is offered by the government. It is a form of insurance that is offered by the government. It is a form of insurance that is offered by the government. It is a form of insurance that is offered by the government. It is a form of insurance that is offered by the government. It is a form of insurance that is offered by the government. It is a form of insurance that is offered by the government. It is a form of insurance that is offered by the government. It is a form of insurance that is offered by the government. It is a form of insurance that is offered by the government. It is a form of insurance that is offered by the government. It is a form of insurance that is offered by the government. It is a form of insurance that is offered by the government. It is a form of insurance that is offered by the government. It is a form of insurance that is offered by the government. It is a form of

In [18]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [19]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [20]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [21]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [22]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])


Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

decreases (1.8 percentage points) 
in the uninsured rate from 2021 to 
2022, which is consistent with the 
decrease in the uninsured rates 
in both South Carolina and North 
Carolina.36 The uninsured rate in 14 
metropolitan areas did not statisti -
cally change between 2021 and 
2022.
34 These most populous metropolitan 
areas had the highest uninsured rates in 
2022 and 2021. Refer to < www.census.
gov/content/dam/Census/library/
publications/2022/acs/acsbr-013.pdf >.
35 When comparing metropolitan areas 
from 2021 to 2022, we used the 25 most 
populous metropolitan areas in 2022 as 
the basis for comparison. However, none of 
the 25 most populous metropolitan areas 
changed from 2021 to 2022; only the order 
of their population ranking within the 25 
metro areas did.
36 The changes in the uninsured 
rates from 2021 to 2022 in Dallas-Fort 
Worth-Arlington, TX 